In [1]:
# prompt: Load the dataset Ehtisham1328/urdu-idioms-with-english-translation from huggingface

!pip install datasets
from datasets import load_dataset

dataset = load_dataset("Ehtisham1328/urdu-idioms-with-english-translation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['محنت کامیابی کی کنجی ہے۔', 'Hard work is the key to success.'],
        num_rows: 2111
    })
})


In [3]:
english = dataset['train']['Hard work is the key to success.']
urdu = dataset['train']['محنت کامیابی کی کنجی ہے۔']

In [4]:
# prompt: write a function to evaluate a model on this dataset (given Urdu, evaluate English translate)

from datasets import load_metric

bleu = load_metric("bleu")


<ipython-input-4-43abd55145a7>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu = load_metric("bleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [5]:
def evaluate_model(model, tokenizer, dataset, samples=5):
    urdus = urdu[:samples]
    englishs = english[:samples]

    model = model.to('cuda')

    urdus = [f'Translate to english: {u}' for u in urdus]
    batch = tokenizer(urdus, return_tensors="pt", padding=True).to('cuda')
    gen = model.generate(**batch)
    predictions = tokenizer.batch_decode(gen, skip_special_tokens=True)
    predictions = [[p] for p in predictions]
    references = [[e] for e in englishs]
    bleu_score = bleu.compute(predictions=predictions, references=references)
    return bleu_score

In [6]:
!pip install --quiet sacremoses

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ur-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ur-en")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/848k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/816k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [8]:
bleu_score = evaluate_model(model, tokenizer, dataset)
print("BLEU score:", bleu_score["bleu"])

ValueError: Got a string but expected a list instead: 'The courage of men is aided by God.'